In [ ]:

import tensorflow as tf
configuration = tf.compat.v1.ConfigProto()
configuration.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=configuration)

from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.optimizers import SGD, Adam, schedules
from tensorflow.keras import utils
from tensorflow.keras.regularizers import l2
import keras

import numpy as np
import h5py
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
!pip install --upgrade tables

     |████████████████████████████████| 4.3MB 5.7MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4


In [ ]:
from numpy.random import seed
seed(1)
tf.random.set_seed(2)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
folder = "/content/drive/My Drive/"

reread = pd.read_hdf(folder+"labeled_data_pickle4.hdf5", key='FR')
countries = ['AT', 'BE', 'BG', 'CY', 'CZ', 'DE', 'DK', 'EE', 'EL', 'ES', 'HR', 'HU', 'IE', 'LT', 'LU', 'LV', 'MT', 'NL', 'PL', 'PT', 'RO', 'SE', 'SI', 'SK', 'UK', 'IT'] #ES FR
for country in countries:
    temppd = pd.read_hdf(folder+"labeled_data_pickle4.hdf5", key=country)
    reread = pd.concat((reread, temppd), ignore_index = True)
temppd = None
print('done')

done


In [ ]:
folder = "/content/drive/My Drive/"
tempdf = pd.read_csv(folder+"LUCAS_Topsoil_2015_20200323.csv" ,usecols=["Point_ID", "LC1_Desc"])
reread = pd.merge(reread, tempdf, on='Point_ID', how='left')

all_landscapes = reread["LC1_Desc"].unique()

print(len(reread))




20559


In [ ]:
filtered_df = pd.DataFrame(columns=reread.columns)

for landscape in all_landscapes:
  temppd  = reread.loc[reread['LC1_Desc'] == landscape] #common wheat
  Q1 = temppd['OC'].quantile(0.25)
  Q3 = temppd['OC'].quantile(0.75)
  IQR = Q3 - Q1
  temppd = temppd[temppd['OC'] < Q3 + IQR * 1.5]
  temppd = temppd[temppd['OC'] > Q1 - IQR * 1.5]
  filtered_df = filtered_df.append(temppd)

reread = filtered_df
filtered_df = None
print(len(reread))



19097


In [ ]:
X = np.array(list(reread['spectogram'].values))
y = reread['OC_state'].values
reread = None

In [ ]:
lb = LabelBinarizer()
y = lb.fit_transform(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
X = None
y = None

In [ ]:
reg = l2(0.0005)
init="he_normal"
chanDim = -1

In [ ]:
opts = ["Ones", "GlorotNormal", 
        "GlorotUniform", "Orthogonal", "Constant", "VarianceScaling", "lecun_uniform", "he_uniform", "LecunNormal"]  #TruncatedNormal

for optie in opts:
  init = optie

  model = Sequential()
  model.add(Conv2D(32, (7, 7), strides=(2, 2), padding="valid",
              kernel_initializer=init, kernel_regularizer=reg,
              input_shape=(217,335,3)))
  # here we stack two CONV layers on top of each other where
  # each layerswill learn a total of 32 (3x3) filters
  model.add(Conv2D(32, (3, 3), padding="same",
      kernel_initializer=init, kernel_regularizer=reg))
  model.add(Activation("relu"))
  model.add(BatchNormalization(axis=chanDim))
  model.add(Conv2D(32, (3, 3), strides=(2, 2), padding="same",
      kernel_initializer=init, kernel_regularizer=reg))
  model.add(Activation("relu"))
  model.add(BatchNormalization(axis=chanDim))
  model.add(Dropout(0.25))
  # stack two more CONV layers, keeping the size of each filter
  # as 3x3 but increasing to 64 total learned filters
  model.add(Conv2D(64, (3, 3), padding="same",
      kernel_initializer=init, kernel_regularizer=reg))
  model.add(Activation("relu"))
  model.add(BatchNormalization(axis=chanDim))
  model.add(Conv2D(64, (3, 3), strides=(2, 2), padding="same",
      kernel_initializer=init, kernel_regularizer=reg))
  model.add(Activation("relu"))
  model.add(BatchNormalization(axis=chanDim))
  model.add(Dropout(0.25))
  # increase the number of filters again, this time to 128
  model.add(Conv2D(128, (3, 3), padding="same",
      kernel_initializer=init, kernel_regularizer=reg))
  model.add(Activation("relu"))
  model.add(BatchNormalization(axis=chanDim))


  # fully-connected layer
  model.add(Flatten())
  model.add(Dense(512, kernel_initializer=init))
  model.add(Activation("selu"))
  model.add(BatchNormalization())
  model.add(Dropout(0.5))

  # softmax classifier
  model.add(Dense((len(y_train[0]))))
  model.add(Activation("softmax"))

  model.compile(loss='categorical_crossentropy', metrics='accuracy', optimizer="adamax")
  history = model.fit(X_train, y_train,
                  batch_size=64,
                  epochs=8,
                  verbose=1, shuffle=True)

  score = model.evaluate(X_test, y_test)
  print(optie)
  print('Test score:', score[0])
  print('Test accuracy:', score[1])

Epoch 1/8
101/224 [============>.................] - ETA: 14:41 - loss: 75.9225 - accuracy: 0.1859

KeyboardInterrupt: ignored

In [ ]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.9)
opt = keras.optimizers.SGD(learning_rate=lr_schedule)




model.compile(loss='categorical_crossentropy', metrics='accuracy', optimizer="adamax")
history = model.fit(X_train, y_train,
                 batch_size=64,
                 epochs=8,
                 verbose=1, shuffle=True)

Epoch 1/8
224/224 [==============================] - 41s 178ms/step - loss: 3.2380 - accuracy: 0.3818
Epoch 2/8
224/224 [==============================] - 40s 180ms/step - loss: 1.6248 - accuracy: 0.5870
Epoch 3/8
224/224 [==============================] - 40s 179ms/step - loss: 1.3761 - accuracy: 0.6174
Epoch 4/8
224/224 [==============================] - 41s 181ms/step - loss: 1.3118 - accuracy: 0.6286
Epoch 5/8
224/224 [==============================] - 40s 180ms/step - loss: 1.2530 - accuracy: 0.6264
Epoch 6/8
224/224 [==============================] - 40s 180ms/step - loss: 1.2233 - accuracy: 0.6440
Epoch 7/8
224/224 [==============================] - 40s 180ms/step - loss: 1.1809 - accuracy: 0.6478
Epoch 8/8
224/224 [==============================] - 40s 180ms/step - loss: 1.1364 - accuracy: 0.6662


In [ ]:
score = model.evaluate(X_test, y_test)
print('Test score:', score[0])
print('Test accuracy:', score[1])

150/150 [==============================] - 4s 28ms/step - loss: 1.2281 - accuracy: 0.6599
Test score: 1.228102207183838
Test accuracy: 0.6598953008651733


In [ ]:
#Test score: 0.9720392227172852
#Test accuracy: 0.6349738240242004

#Test score: 1.0995421409606934
#Test accuracy: 0.6544502377510071


In [ ]:
opts = ["random_normal", "RandomUniform", "TruncatedNormal", "Zeros", "Ones", "GlorotNormal", 
        "GlorotUniform", "Orthogonal", "Constant", "VarianceScaling", "lecun_uniform", "he_uniform", "LecunNormal"]
for op in opts:
  model = Sequential()
  model.add(Conv2D(32, (7, 7), strides=(2, 2), padding="valid",
                kernel_initializer=op, kernel_regularizer=reg,
                input_shape=(217,335,3)))
  print(op)

random_normal
RandomUniform
TruncatedNormal
Zeros
Ones
GlorotNormal
GlorotUniform
Orthogonal
Constant
VarianceScaling
lecun_uniform
he_uniform
LecunNormal
